In [ ]:
import os
import time
import unicodedata
from collections import Counter
from memory_profiler import memory_usage

def normalize_text(text):
    normalized = unicodedata.normalize('NFD', text)
    filtered = ''.join(c for c in normalized if unicodedata.category(c) != 'Mn')
    only_alpha = ''.join(filter(str.isalpha, filtered))
    return only_alpha.lower()

def calculate_letter_frequency(file_path):
    letter_frequency = Counter()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            normalized_text = normalize_text(line)
            letter_frequency.update(Counter(normalized_text))
    total_letters = sum(letter_frequency.values())
    letter_frequency = {letter: freq / total_letters for letter, freq in letter_frequency.items()}
    
    return letter_frequency, total_letters

def measure_performance(file_path, file_size, output_dir):
    start_time = time.time()
    start_cpu = time.process_time()
    mem_usage = memory_usage((calculate_letter_frequency, (file_path,)), interval=0.1)
    letter_frequency, total_letters = calculate_letter_frequency(file_path)
    end_time = time.time()
    end_cpu = time.process_time()
    
    execution_time_ms = (end_time - start_time) * 1000
    cpu_time_ms = (end_cpu - start_cpu) * 1000
    max_memory_bytes = max(mem_usage) * (1024 ** 2)
    
    results = f"Dimensione file: {file_size}\nTempo di esecuzione: {execution_time_ms:.2f} ms\nTempo di CPU: {cpu_time_ms:.2f} ms\nMemoria massima utilizzata: {max_memory_bytes:.0f} bytes\n\n"
    
    specific_output_dir = os.path.join(output_dir, file_size)
    os.makedirs(specific_output_dir, exist_ok=True)
    
    execution_output_file = f"PythonExecution_{file_size}.txt"
    execution_full_output_path = os.path.join(specific_output_dir, execution_output_file)
    with open(execution_full_output_path, "w") as file:
        file.write(results)
    
    total_letters_file = f"count.txt"
    total_letters_path = os.path.join(specific_output_dir, total_letters_file)
    with open(total_letters_path, "w") as file:
        file.write(f"Conteggio totale delle lettere: {total_letters}\n")
    
    frequency_file = f"frequency.txt"
    frequency_path = os.path.join(specific_output_dir, frequency_file)
    with open(frequency_path, "w") as file:
        for letter, freq in letter_frequency.items():
            file.write(f"{letter}: {freq}\n")
    
    print(f"Risultati salvati in {execution_full_output_path}")
    print(f"Conteggio totale delle lettere salvato in {total_letters_path}")
    print(f"Frequenza delle lettere salvata in {frequency_path}")

output_directory = "../Resources/PythonExecution/Output/"

files_to_process = [
    ("file_2KB", "../Resources/Performance/Input/file_2KB.txt"),
    ("file_2MB", "../Resources/Performance/Input/file_2MB.txt"),
    ("file_2GB", "../../file_2GB.txt")
]

for file_size, file_path in files_to_process:
    measure_performance(file_path, file_size, output_directory)